In [ ]:
import torch
import numpy as np
import torch.nn as nn

class onering_conv_layer(nn.Module):
    """The convolutional layer on icosahedron discretized sphere using 
    1-ring filter
    
    Parameters:
            in_feats (int) - - input features/channels
            out_feats (int) - - output features/channels
            
    Input: 
        N x in_feats tensor
    Return:
        N x out_feats tensor
    """  
    def __init__(self, in_feats, out_feats, neigh_orders, neigh_indices=None, neigh_weights=None):
        super(onering_conv_layer, self).__init__()

        self.in_feats = in_feats
        self.out_feats = out_feats
        self.neigh_orders = neigh_orders
        self.pool = pool_layer(self.neigh_orders, pooling_type='mean')
        self.weight = nn.Linear(7 * in_feats, out_feats)
        # self.norm = nn.BatchNorm1d(out_feats, momentum=0.15, affine=True, track_running_stats=False)
        # self.relu = nn.LeakyReLU(0.2, inplace=True)
        # self.dropout = nn.Dropout(0.7)
        
    def forward(self, x):
       
        mat = x[self.neigh_orders].view(len(x), 7*self.in_feats)
                
        out_features = self.weight(mat)
        # out_features = self.norm(out_features)
        # out_features = self.relu(out_features)
        # out_features = self.pool(out_features)
        # out_features = self.dropout(out_features)
        
        return out_features
    
class tworing_conv_layer(nn.Module):
    """The convolutional layer on icosahedron discretized sphere using 
    2-ring filter
    
    Parameters:
            in_feats (int) - - input features/channels
            out_feats (int) - - output features/channels
            
    Input: 
        N x in_feats tensor
    Return:
        N x out_feats tensor
    """  
    def __init__(self, in_feats, out_feats, neigh_orders):
        super(tworing_conv_layer, self).__init__()

        self.in_feats = in_feats
        self.out_feats = out_feats
        self.neigh_orders = neigh_orders
        
        self.weight = nn.Linear(19 * in_feats, out_feats)
        
    def forward(self, x):
       
        mat = x[self.neigh_orders].view(len(x), 19*self.in_feats)

        out_features = self.weight(mat)
        return out_features
    

class pool_layer(nn.Module):
    """
    The pooling layer on icosahedron discretized sphere using 1-ring filter
    
    Input: 
        N x D tensor
    Return:
        ((N+6)/4) x D tensor
    
    """  

    def __init__(self, neigh_orders, pooling_type='mean'):
        super(pool_layer, self).__init__()

        self.neigh_orders = neigh_orders
        self.pooling_type = pooling_type
        
    def forward(self, x):
       
        num_nodes = int((x.size()[0]+6)/4)
        feat_num = x.size()[1]
        x = x[self.neigh_orders[0:num_nodes*7]].view(num_nodes, feat_num, 7)
        if self.pooling_type == "mean":
            x = torch.mean(x, 2)
        if self.pooling_type == "max":
            x = torch.max(x, 2)
            assert(x[0].size() == torch.Size([num_nodes, feat_num]))
            return x[0], x[1]
        
        assert(x.size() == torch.Size([num_nodes, feat_num]))
                
        return x
    
class upconv_layer(nn.Module):
    """
    The transposed convolution layer on icosahedron discretized sphere using 1-ring filter
    
    Input: 
        N x in_feats, tensor
    Return:
        ((Nx4)-6) x out_feats, tensor
    
    """  

    def __init__(self, in_feats, out_feats, upconv_top_index, upconv_down_index):
        super(upconv_layer, self).__init__()

        self.in_feats = in_feats
        self.out_feats = out_feats
        self.upconv_top_index = upconv_top_index
        self.upconv_down_index = upconv_down_index
        self.weight = nn.Linear(in_feats, 7 * out_feats)
        
    def forward(self, x):
       
        raw_nodes = x.size()[0]
        new_nodes = int(raw_nodes*4 - 6)
        x = self.weight(x)
        x = x.view(len(x) * 7, self.out_feats)
        x1 = x[self.upconv_top_index]
        assert(x1.size() == torch.Size([raw_nodes, self.out_feats]))
        x2 = x[self.upconv_down_index].view(-1, self.out_feats, 2)
        x = torch.cat((x1,torch.mean(x2, 2)), 0)
        assert(x.size() == torch.Size([new_nodes, self.out_feats]))
        return x

In [ ]:
import scipy.io as sio

def my_mat_Get_2ring_neighs_order():
    neigh_orders_2ring_40962 = my_mat_get_2ring_neighs_order('neighbour_indices_matlab/adj_mat_order_2ring_40962.mat')
    neigh_orders_2ring_10242 = my_mat_get_2ring_neighs_order('neighbour_indices_matlab/adj_mat_order_2ring_10242.mat')
    neigh_orders_2ring_2562 = my_mat_get_2ring_neighs_order('neighbour_indices_matlab/adj_mat_order_2ring_2562.mat')
    neigh_orders_2ring_642 = my_mat_get_2ring_neighs_order('neighbour_indices_matlab/adj_mat_order_2ring_642.mat')
    neigh_orders_2ring_162 = my_mat_get_2ring_neighs_order('neighbour_indices_matlab/adj_mat_order_2ring_162.mat')
    neigh_orders_2ring_42 = my_mat_get_2ring_neighs_order('neighbour_indices_matlab/adj_mat_order_2ring_42.mat')
    
    return neigh_orders_2ring_40962, neigh_orders_2ring_10242, neigh_orders_2ring_2562, neigh_orders_2ring_642, neigh_orders_2ring_162, neigh_orders_2ring_42

def my_mat_get_2ring_neighs_order(order_path):
    adj_mat_order = sio.loadmat(order_path)
    adj_mat_order = adj_mat_order['adj_mat_order_2ring']
    neigh_orders = np.zeros((len(adj_mat_order), 19))
    neigh_orders[:,0:18] = adj_mat_order-1
    neigh_orders[:,18] = np.arange(len(adj_mat_order))
    neigh_orders = np.ravel(neigh_orders).astype(np.int64)
    
    return neigh_orders

neigh_orders_2ring_40962, neigh_orders_2ring_10242, neigh_orders_2ring_2562, neigh_orders_2ring_642, neigh_orders_2ring_162, neigh_orders_2ring_42 = my_mat_Get_2ring_neighs_order()
neigh_orders_1ring_40962 = my_mat_get_2ring_neighs_order('neighbour_indices_matlab/adj_mat_order_40962.mat')
print(neigh_orders_2ring_40962.shape)

In [ ]:
import matplotlib.pyplot as plt

################################################################
""" hyper-parameters """
# cuda = torch.device('cpu')
device = torch.device("cuda:0")
batch_size = 1
model_name = 'Unet_infant'  # 'Unet_infant', 'Unet_18', 'Unet_2ring', 'Unet_repa', 'fcn', 'SegNet', 'SegNet_max'
up_layer = 'upsample_interpolation' # 'upsample_interpolation', 'upsample_fixindex' 
in_channels = 1
out_channels = 1
learning_rate = 0.001
momentum = 0.99
weight_decay = 0.0001
fold = 1 # 1,2,3 
################################################################

data_file = 'data.npy'
label_file = 'labels.npy'

# 从.npy文件中加载数据
data_array = np.load(data_file)
train_data = data_array[0:400,:]
val_data = data_array[400:,:]
print(data_array.shape,train_data.shape)
label_array = np.load(label_file)
train_label = label_array[0:400]
val_label = label_array[400:]

neighbors_path = 'C:/Users/DELL/Desktop/kaiti/Spherical_U-Net/neigh_indices/adj_mat_order_10242.mat'

def get_neighs_order(neighbors_path):
    adj_mat_order = sio.loadmat(neighbors_path)
    adj_mat_order = adj_mat_order['adj_mat_order']
    neigh_orders = np.zeros((len(adj_mat_order), 7))
    neigh_orders[:,0:6] = adj_mat_order-1
    neigh_orders[:,6] = np.arange(len(adj_mat_order))
    neigh_orders = np.ravel(neigh_orders).astype(np.int64)
    
    return neigh_orders

class BrainSphere(torch.utils.data.Dataset):

    def __init__(self, data, label):
        self.data = data
        self.labels = label
        
    def __len__(self):
        return len(self.labels)


    def __getitem__(self, idx):
        sample = {
            'data': torch.from_numpy(self.data[idx]),
            'label': torch.tensor(self.labels[idx])
        }
        return sample


class Spherical_CNN(nn.Module):
    """Define the Spherical UNet structure

    """    
    def __init__(self, in_ch, out_ch):
        """ Initialize the Spherical UNet.

        Parameters:
            in_ch (int) - - input features/channels
            out_ch (int) - - output features/channels
        """
        super(Spherical_CNN, self).__init__()

        #neigh_indices_10242, neigh_indices_2562, neigh_indices_642, neigh_indices_162, neigh_indices_42 = Get_indices_order()
        #neigh_orders_10242, neigh_orders_2562, neigh_orders_642, neigh_orders_162, neigh_orders_42, neigh_orders_12 = Get_neighs_order()
        
        neigh_orders = get_neighs_order(neighbors_path)

        
        conv_layer = onering_conv_layer

        self.block = nn.Sequential(
            conv_layer(in_ch, 64, neigh_orders),
            nn.BatchNorm1d(64, momentum=0.15, affine=True, track_running_stats=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.7),
            conv_layer(64, 128, neigh_orders),
            nn.BatchNorm1d(128, momentum=0.15, affine=True, track_running_stats=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 1)
        )
        self.linear = nn.Linear(10242, 1)
            

                
        
    def forward(self, x):
        # batch norm version
        x = self.block(x)
        x = x.squeeze()
        # print(x.shape) 
        x = self.linear(x)
        # x = self.outc(x)
        
        return x
    
# train_dataset = BrainSphere(data_file, label_file)
train_dataset = BrainSphere(train_data, train_label)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=False, pin_memory=True)

val_dataset = BrainSphere(val_data, val_label)

val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False, pin_memory=True)
# for batch in train_dataloader:
#     data, labels = batch['data'], batch['label']
#     # 在这里可以进行你的训练或其他操作
#     print(f"Batch Data Shape: {data.shape}, Batch Label Shape: {labels.shape}")

model = Spherical_CNN(in_ch=1, out_ch=out_channels)

print("{} paramerters in total".format(sum(x.numel() for x in model.parameters())))
model.to(device)
criterion = nn.MSELoss()
val_criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler1 = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.5)


def train_step(data, target):
    # model.train()
    data, target = data.to(device), target.to(device)

    prediction = model(data)
    # print(prediction)
    loss = criterion(prediction, target)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item(),prediction


# # train_dice = [0, 0, 0, 0, 0]
print('length of dataloader:', len(train_dataloader))
loss_list = []
val_loss_list = []
for epoch in range(100):
    total_loss = 0
    pre_list = []
    label_list = []
    model.train() 

    if epoch % 20 == 9:
        scheduler1.step()
    
    for batch_idx, train_data in enumerate(train_dataloader):
        data = train_data['data']
        data = torch.squeeze(data).unsqueeze(1)
        target = train_data['label']
        target = torch.tensor(target, dtype=torch.float32)
        # print(target)
        loss,pre = train_step(data, target)
        pre_list.append(pre.item())
        label_list.append(target.item())
        total_loss = total_loss + loss
    # print(total_loss/len(train_dataloader))
    loss_list.append(total_loss/len(train_dataloader))
    # print(pre_list)
    # print(label_list)

    model.eval()  # 切换到评估模式
        
    with torch.no_grad():  # 禁用梯度计算，因为在验证阶段我们不需要反向传播
        val_loss_total = 0
        val_pre_list = []
        val_label_list = []
        
        for batch_idy, val_dataset in enumerate(val_dataloader):
            val_data = val_dataset['data']
            val_data = torch.squeeze(val_data).unsqueeze(1)
            val_target = val_dataset['label']
            val_target = torch.tensor(val_target, dtype=torch.float32)
            data, target = val_data.to(device), val_target.to(device)

            prediction = model(data)
            # print(prediction)
            val_loss = val_criterion(prediction, target)
            val_loss = val_loss.item()
            


            val_pre_list.append(prediction.item())
            val_label_list.append(val_target.item())
            val_loss_total = val_loss_total + val_loss
        
        # 计算平均验证损失
        avg_val_loss = val_loss_total / len(val_dataloader)
        val_loss_list.append(val_loss_total / len(val_dataloader))
        print(avg_val_loss)
        correlation_coefficient = np.corrcoef(val_pre_list, val_label_list)[0, 1]
        print(f'Correlation Coefficient: {correlation_coefficient}')
        # print(val_pre_list)
        # print(val_label_list)



plt.plot(loss_list)
# val_loss_list2 = []
# for i in val_loss_list:
#     val_loss_list2.append(i.item())

plt.plot(val_loss_list)